In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210410.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice]
0,06/04/2021 17:37:46,King Kong (1933),Nénette (2010),Outbreak (1995),Congo (1995),Max Mon Amour (1986),Godzilla vs. Kong (2021),MVP: Most Vertical Primate (2001),"WELCOME TO BAD PLACE, USA!",I LOVE TO KISS MY HOMIES,BUNGA BUNGA,OOPS ALL JOHN WOO,GOOD FOR HER.,NUTTY PROFESSOR,SHARK GAWR >:V,[Default to Nicolas Cage]
1,06/04/2021 17:57:43,Godzilla vs. Kong (2021),Nénette (2010),Outbreak (1995),MVP: Most Vertical Primate (2001),King Kong (1933),Congo (1995),Max Mon Amour (1986),GOOD FOR HER.,[Default to Nicolas Cage],"WELCOME TO BAD PLACE, USA!",OOPS ALL JOHN WOO,I LOVE TO KISS MY HOMIES,NUTTY PROFESSOR,SHARK GAWR >:V,BUNGA BUNGA
2,06/04/2021 17:59:36,Godzilla vs. Kong (2021),Max Mon Amour (1986),Nénette (2010),MVP: Most Vertical Primate (2001),Outbreak (1995),Congo (1995),NaN,OOPS ALL JOHN WOO,"WELCOME TO BAD PLACE, USA!",I LOVE TO KISS MY HOMIES,SHARK GAWR >:V,NUTTY PROFESSOR,[Default to Nicolas Cage],GOOD FOR HER.,BUNGA BUNGA
3,06/04/2021 18:16:21,Congo (1995),MVP: Most Vertical Primate (2001),Outbreak (1995),Godzilla vs. Kong (2021),King Kong (1933),Max Mon Amour (1986),Nénette (2010),"WELCOME TO BAD PLACE, USA!",I LOVE TO KISS MY HOMIES,[Default to Nicolas Cage],BUNGA BUNGA,OOPS ALL JOHN WOO,SHARK GAWR >:V,GOOD FOR HER.,NUTTY PROFESSOR
4,06/04/2021 20:02:10,Godzilla vs. Kong (2021),MVP: Most Vertical Primate (2001),Nénette (2010),NaN,NaN,NaN,NaN,I LOVE TO KISS MY HOMIES,SHARK GAWR >:V,"WELCOME TO BAD PLACE, USA!",BUNGA BUNGA,GOOD FOR HER.,NUTTY PROFESSOR,OOPS ALL JOHN WOO,[Default to Nicolas Cage]
5,07/04/2021 05:28:34,Congo (1995),Godzilla vs. Kong (2021),MVP: Most Vertical Primate (2001),Max Mon Amour (1986),Outbreak (1995),NaN,NaN,NUTTY PROFESSOR,"WELCOME TO BAD PLACE, USA!",[Default to Nicolas Cage],OOPS ALL JOHN WOO,I LOVE TO KISS MY HOMIES,SHARK GAWR >:V,NaN,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['King Kong (1933)', 'Nénette (2010)', 'Outbreak (1995)',
       'Congo (1995)', 'Max Mon Amour (1986)', 'Godzilla vs. Kong (2021)',
       'MVP: Most Vertical Primate (2001)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'King Kong (1933)': <Candidate('King Kong (1933)')>,
 'Nénette (2010)': <Candidate('Nénette (2010)')>,
 'Outbreak (1995)': <Candidate('Outbreak (1995)')>,
 'Congo (1995)': <Candidate('Congo (1995)')>,
 'Max Mon Amour (1986)': <Candidate('Max Mon Amour (1986)')>,
 'Godzilla vs. Kong (2021)': <Candidate('Godzilla vs. Kong (2021)')>,
 'MVP: Most Vertical Primate (2001)': <Candidate('MVP: Most Vertical Primate (2001)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(King Kong (1933), Nénette (2010), Outbreak (1995), Congo (1995), Max Mon Amour (1986), Godzilla vs. Kong (2021), MVP: Most Vertical Primate (2001))>,
 <Ballot(Godzilla vs. Kong (2021), Nénette (2010), Outbreak (1995), MVP: Most Vertical Primate (2001), King Kong (1933), Congo (1995), Max Mon Amour (1986))>,
 <Ballot(Godzilla vs. Kong (2021), Max Mon Amour (1986), Nénette (2010), MVP: Most Vertical Primate (2001), Outbreak (1995), Congo (1995))>,
 <Ballot(Congo (1995), MVP: Most Vertical Primate (2001), Outbreak (1995), Godzilla vs. Kong (2021), King Kong (1933), Max Mon Amour (1986), Nénette (2010))>,
 <Ballot(Godzilla vs. Kong (2021), MVP: Most Vertical Primate (2001), Nénette (2010))>,
 <Ballot(Congo (1995), Godzilla vs. Kong (2021), MVP: Most Vertical Primate (2001), Max Mon Amour (1986), Outbreak (1995))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

FINAL RESULT
Candidate                            Votes  Status
---------------------------------  -------  --------
Godzilla vs. Kong (2021)                 3  Elected
Congo (1995)                             2  Rejected
King Kong (1933)                         1  Rejected
Nénette (2010)                           0  Rejected
MVP: Most Vertical Primate (2001)        0  Rejected
Max Mon Amour (1986)                     0  Rejected
Outbreak (1995)                          0  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                     Votes  Status
--------------------------  -------  --------
WELCOME TO BAD PLACE, USA!        2  Hopeful
I LOVE TO KISS MY HOMIES          1  Hopeful
OOPS ALL JOHN WOO                 1  Hopeful
GOOD FOR HER.                     1  Hopeful
NUTTY PROFESSOR                   1  Rejected
[Default to Nicolas Cage]         0  Rejected
SHARK GAWR >:V                    0  Rejected
BUNGA BUNGA                       0  Rejected

ROUND 2
Candidate                     Votes  Status
--------------------------  -------  --------
WELCOME TO BAD PLACE, USA!        3  Hopeful
OOPS ALL JOHN WOO                 1  Hopeful
I LOVE TO KISS MY HOMIES          1  Rejected
GOOD FOR HER.                     1  Rejected
NUTTY PROFESSOR                   0  Rejected
[Default to Nicolas Cage]         0  Rejected
SHARK GAWR >:V                    0  Rejected
BUNGA BUNGA                       0  Rejected

FINAL RESULT
Candidate                     Votes  Status
---------------